In [ ]:
!pip install groq
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 857.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import re
import json

import groq
from groq import Groq
from google.colab import userdata
import unidecode

from google.colab import userdata, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
client = Groq(api_key=userdata.get('groq.key'))
def send_message(message):
    '''
    Dada uma mensagem, manda pelo groq para o llama3-70b-8192 e retorna a resposta do LLM.

    Input: message (str): A mensagem a ser enviada ao LLM.

    Observações: Note que para minimizar as incertezas do processo e buscando maximizar a qualidade da resposta,
    estamos usando a temperatura 0 e top_p 1.

    Output: Uma string contendo a saída do Llama3
    '''
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": message}],
        model='llama3-70b-8192',
        temperature = 0,
        top_p = 1
    )
    return chat_completion.choices[0].message.content

In [ ]:
first_query = send_message('''

Escreva as 100 possíveis pesquisas mais frequentes no Google sobre o direito no Brasil, uma por linha.

Seguem alguns exemplos já obtidos no site para referência:

1. Direito do trabalho
2. Como consultar processo
3. Porque meu nome esta no diario oficial
4. Justiça gratuita servidor publico
''')

In [ ]:
base = {'buscas': re.findall(r'\d+\. (.*)',first_query)}

In [ ]:
first_clean = list(map(lambda x: unidecode.unidecode(x), re.findall(r'\d+\. (.*)',first_query)))
base = {'buscas': first_clean}

In [ ]:
with open('./data/buscas.json', 'w') as jsonfile:
  json.dump(base, jsonfile)

In [ ]:
unidecode.unidecode('alto lá')

'alto la'